In [38]:
import numpy as np
import pandas as pd
from datetime import datetime
import json

def calculate_dynamic_beer_price(df, min_adjustment=-0.15, max_adjustment=0.40, noise_level=0.02):
    """
    Calculates dynamic beer price based on behavioral and situational factors.
    """
    # Behavioral Markups
    user_history_markup = {
        'New': 0.98, 'Frequent': 1.02, 'Premium': 1.03, 'VIP': 1.04, 'Occasional': 1.00, 'Lapsed': 0.97
    }
    age_group_markup = {
        '18-24': 1.02, '25-34': 1.03, '35-44': 1.02, '45-54': 1.01, '55+': 0.99
    }
    device_type_markup = {
        'Mobile': 1.02, 'Desktop': 1.00, 'Tablet': 1.01, 'Smart TV': 1.00
    }
    os_type_markup = {
        'iOS': 1.03, 'Android': 1.02, 'Windows': 1.00, 'MacOS': 1.01
    }
    screen_size_markup = {
        'Small': 0.98, 'Medium': 1.00, 'Large': 1.02
    }
    competitor_pricing_markup = {
        'Lower': 0.98, 'Similar': 1.00, 'Higher': 1.02
    }
    regional_demand_markup = {
        'Low': 0.98, 'Medium': 1.01, 'High': 1.04
    }
    customer_rating_markup = {
        'Low': 0.97, 'Medium': 1.00, 'High': 1.03
    }
    payday_proximity_markup = {
        'Far': 0.99, 'Near': 1.02
    }
    tourism_level_markup = {
        'Low': 0.98, 'Medium': 1.01, 'High': 1.04
    }
    product_marketing_status_markup = {
        'Low': 0.98, 'Medium': 1.01, 'High': 1.03
    }
    def get_time_day_markup(hour, day):
        time_markup = 0.98 if hour < 12 else (1.00 if hour < 16 else 1.02)
        day_markup = {'Monday': 0.98, 'Tuesday': 0.99, 'Wednesday': 1.00, 'Thursday': 1.01, 'Friday': 1.04, 'Saturday': 1.05, 'Sunday': 1.02}
        happy_hour_factor = 0.98 if 16 <= hour < 19 else 1.00
        return time_markup * day_markup.get(day, 1.00) * happy_hour_factor

    demand_markup = {'Very Low': 0.95, 'Low': 0.97, 'Medium': 1.00, 'High': 1.02, 'Very High': 1.04}
    event_markup = {'None': 1.00, 'Sports Event': 1.05, 'Festival': 1.04, 'Holiday': 1.04, 'Concert': 1.06}

    def get_weather_markup(row):
        temp_factor = 1 + (row.get('Temperature', 20) - 20) * 0.005
        weather_factors = {'Sunny': 1.02, 'Rainy': 0.98, 'Cloudy': 0.99, 'Snow': 1.02, 'Clear': 1.01}
        return temp_factor * weather_factors.get(row.get('WeatherCondition', 'Clear'), 1.00)

    inventory_level_markup = {'Low': 1.04, 'Medium': 1.02, 'High': 0.98}
    city_type_markup = {'Urban': 1.03, 'Suburban': 1.02, 'Rural': 0.98}
    weekday_type_markup = {'Weekday': 1.00, 'Weekend': 1.03}
    day_type_markup = {'Regular': 1.00, 'Holiday': 1.03}

    base_selling_price=df['BaseSellingPrice']
    max_price = base_selling_price * (1 + max_adjustment)
    min_price = base_selling_price * (1 + min_adjustment)
    
    base_price = df['BaseSellingPrice']
    base_price *= df['UserHistory'].map(user_history_markup)
    base_price *= df['AgeGroup'].map(age_group_markup)
    base_price *= df['DeviceType'].map(device_type_markup)
    base_price *= df['OSType'].map(os_type_markup)
    base_price *= df['ScreenSize'].map(screen_size_markup)
    base_price *= df['CompetitorPricing'].map(competitor_pricing_markup)
    base_price *= df['RegionalDemand'].map(regional_demand_markup)
    base_price *= df['CustomerRating'].map(customer_rating_markup)
    base_price *= df['PaydayProximity'].map(payday_proximity_markup)
    base_price *= df['TourismLevel'].map(tourism_level_markup)
    base_price *= df['ProductMarketingStatus'].map(product_marketing_status_markup)
    base_price *= df.apply(lambda x: get_time_day_markup(datetime.now().hour, x['DayOfWeek']), axis=1)
    base_price *= df['DemandIndex'].map(demand_markup)
    base_price *= df['SpecialOccasion'].map(event_markup)
    base_price *= df.apply(get_weather_markup, axis=1)
    base_price *= df['InventoryLevel'].map(inventory_level_markup)
    base_price *= df['CityType'].map(city_type_markup)
    base_price *= df['WeekdayType'].map(weekday_type_markup)
    base_price *= df['DayType'].map(day_type_markup)
    # noise = np.random.normal(0, noise_level, len(df))
    # base_price *= (1 + noise)
    price = np.clip(float(base_price.iloc[0]), float(min_price.iloc[0]), float(max_price.iloc[0]))    
    return price
    
example_json = '''[
    {"BaseSellingPrice": 5.0, "UserHistory": "Frequent", "AgeGroup": "25-34", "DeviceType": "Mobile", "OSType": "iOS", "ScreenSize": "Large", "CompetitorPricing": "Similar", "RegionalDemand": "High", "CustomerRating": "High", "PaydayProximity": "Near", "TourismLevel": "Medium", "ProductMarketingStatus": "High", "DayOfWeek": "Friday", "DemandIndex": "Medium", "SpecialOccasion": "None", "Temperature": 22, "WeatherCondition": "Sunny", "InventoryLevel": "Medium", "CityType": "Urban", "WeekdayType": "Weekday", "DayType": "Regular"}
]'''
df_example = pd.DataFrame(json.loads(example_json))
 
print(calculate_dynamic_beer_price(df_example))


0    7.0
Name: BaseSellingPrice, dtype: float64
7.0


In [23]:
# import numpy as np
# import pandas as pd
# import tensorflow as tf
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input, Dense, BatchNormalization
# from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import MinMaxScaler
# import joblib

# def generate_dynamic_beer_price_data(n_samples):
#     """
#     Generates independent variables for dynamic beer pricing.
#     Returns a DataFrame with relevant features.
#     """
#     data = {
#         'UserHistory': np.random.choice(['New', 'Frequent', 'Rare'], n_samples),
#         'AgeGroup': np.random.choice(['18-24', '25-34', '35-44', '45-54', '55+'], n_samples),
#         'DeviceType': np.random.choice(['Mobile', 'Desktop', 'Tablet'], n_samples),
#         'OSType': np.random.choice(['iOS', 'Android', 'Windows', 'MacOS'], n_samples),
#         'ScreenSize': np.random.choice(['Small', 'Medium', 'Large'], n_samples),
#         'CompetitorPricing': np.random.uniform(5, 15, n_samples),
#         'RegionalDemand': np.random.choice(['Low', 'Medium', 'High'], n_samples),
#         'CustomerRating': np.random.uniform(1, 5, n_samples),
#         'PaydayProximity': np.random.choice(['Far', 'Near']),
#         'TourismLevel': np.random.choice(['Low', 'Medium', 'High'], n_samples),
#         'ProductMarketingStatus': np.random.choice(['Low', 'Medium', 'High'], n_samples),
#         'DayOfWeek': np.random.choice(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], n_samples),
#         'DemandIndex': np.random.choice(['Low', 'Medium', 'High'], n_samples),
#         'SpecialOccasion': np.random.choice([0, 1], n_samples),
#         'WeatherCondition': np.random.choice(['Sunny', 'Rainy', 'Snowy', 'Cloudy'], n_samples),
#         'InventoryLevel': np.random.choice(['Low', 'Medium', 'High'], n_samples),
#         'CityType': np.random.choice(['Urban', 'Suburban', 'Rural'], n_samples),
#         'WeekdayType': np.random.choice(['Weekday', 'Weekend'], n_samples),
#         'DayType': np.random.choice(['Holiday', 'Workday', 'Special Event'], n_samples),
#         'BaseSellingPrice': np.random.uniform(5, 20, n_samples),
#         'Temperature': np.random.uniform(-5, 35, n_samples),
#         'PaydayProximity': np.random.choice(['Far', 'Near'], n_samples)

#     }
#     return pd.DataFrame(data)

# def create_improved_model(input_dim, learning_rate=0.0007):
#     """Create an improved model architecture for beer price prediction"""
#     main_input = Input(shape=(input_dim,), dtype=tf.float32)
#     normalized = BatchNormalization()(main_input)
#     dense1 = Dense(128, activation='relu')(normalized)
#     dense2 = Dense(64, activation='relu')(dense1)
#     output = Dense(1)(dense2)
#     model = Model(inputs=main_input, outputs=output)
#     model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
#                  loss='mse',
#                  metrics=['mae', 'mse'])
#     return model

# # Generate and prepare data
# print("Generating beer price data...")
# df = generate_dynamic_beer_price_data(10000)

# # Define numerical and categorical features
# num_features = ['BaseSellingPrice', 'Temperature', 'CompetitorPricing', 'CustomerRating']
# cat_features = [
#     'UserHistory', 'AgeGroup', 'DeviceType', 'OSType', 'ScreenSize', 'RegionalDemand',
#     'PaydayProximity', 'TourismLevel', 'ProductMarketingStatus', 'DayOfWeek', 'DemandIndex',
#     'SpecialOccasion', 'WeatherCondition', 'InventoryLevel', 'CityType', 'WeekdayType', 'DayType'
# ]

# # Encode categorical features
# df['DemandIndex'] = df['DemandIndex'].map({'Low': -1, 'Medium': 0, 'High': 1}).astype(np.float32)
# df_encoded = pd.get_dummies(df, columns=[col for col in cat_features if col != 'DemandIndex'], dtype=np.float32)

# feature_names = [col for col in df_encoded.columns if col not in ['Price']]

# # Scale numerical features
# scaler = MinMaxScaler()
# df_encoded[num_features] = scaler.fit_transform(df_encoded[num_features])

# # Calculate target variable: Beer Price
# df_encoded['Price'] = calculate_dynamic_beer_price(df)

# # Save preprocessing objects
# joblib.dump(scaler, 'beer_price_scaler_v2.pkl')
# joblib.dump(feature_names, 'beer_price_feature_names_v2.pkl')

# # Prepare features and target
# X = df_encoded[feature_names].values.astype(np.float32)
# y = df_encoded['Price'].values.astype(np.float32)

# # Split data
# X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
# X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# # Create and train improved model
# model = create_improved_model(X_train.shape[1], learning_rate=0.001)

# callbacks = [
#     EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True),
#     ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=7, min_lr=0.00001)
# ]

# # Train model
# print("\nTraining Beer Price Prediction Model V2...")
# history = model.fit(
#     X_train, y_train,
#     validation_data=(X_valid, y_valid),
#     epochs=200,
#     batch_size=64,
#     callbacks=callbacks,
#     verbose=1
# )

# # Save model and test data
# model.save('beer_price_model_v2.keras')
# np.save('X_test_beer_price_v2.npy', X_test)
# np.save('y_test_beer_price_v2.npy', y_test)


Generating beer price data...

Training Beer Price Prediction Model V2...
Epoch 1/200
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: nan - mae: nan - mse: nan - val_loss: nan - val_mae: nan - val_mse: nan - learning_rate: 0.0010
Epoch 2/200
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: nan - mae: nan - mse: nan - val_loss: nan - val_mae: nan - val_mse: nan - learning_rate: 0.0010
Epoch 3/200
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: nan - mae: nan - mse: nan - val_loss: nan - val_mae: nan - val_mse: nan - learning_rate: 0.0010
Epoch 4/200
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: nan - mae: nan - mse: nan - val_loss: nan - val_mae: nan - val_mse: nan - learning_rate: 0.0010
Epoch 5/200
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: nan - mae: nan - mse: nan - val_loss: nan - val_mae: nan - val_mse: nan - learning_rate: 0.0010
Epoch 6/200
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: nan - mae: nan - mse: nan - val_loss: nan - val_mae: nan - val_mse: nan - learni